In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import joblib
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

In [2]:
from sqlalchemy import create_engine

In [3]:
host = r'127.0.0.1' 
db = r'MSDS610' 
user = r'postgres' 
pw = r'7205458456' 
port = r'5432' 
schema = r'cleaned'

In [4]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [5]:
query = f"SELECT * FROM {schema}.clean_df" 
ObesityData2 = pd.read_sql(query, db_conn)


In [10]:
ObesityData2.to_csv(r"C:\Users\thati\OneDrive\Documentos\Regis\3_Data Engineering\Class_6\ObesityData2.csv", index=False)

I established a connection to the PostgreSQL database using the SQLAlchemy library. The schema used for this connection was set to cleaned, which was part of the MSDS610 database The dataset was saved as a CSV file to a local directory.

Analytical Question: 

What factors are most influential in classifying the level of obesity in individuals?

In [22]:
import pandas as pd
df = pd.read_csv('ObesityData2.csv')


In [23]:
print(df.head())

   Gender   Age  Height  Weight  family_history_with_overweight  FAVC  FCVC  \
0       0  21.0    1.62    64.0                               1     0   2.0   
1       0  21.0    1.52    56.0                               1     0   3.0   
2       1  23.0    1.80    77.0                               1     0   2.0   
3       1  27.0    1.80    87.0                               0     0   3.0   
4       1  22.0    1.78    89.8                               0     0   2.0   

   NCP  CAEC  SMOKE  ...  SCC  FAF  TUE  CALC  MTRANS  NObeyesdad  \
0  3.0     1      0  ...    0  0.0  1.0     0       0           0   
1  3.0     1      1  ...    1  3.0  0.0     1       0           0   
2  3.0     1      0  ...    0  2.0  1.0     2       0           0   
3  3.0     1      0  ...    0  2.0  0.0     2       1           1   
4  1.0     1      0  ...    0  0.0  0.0     1       0           2   

           BMI  Water_Calories_Ratio  CAEC_IMC_Relationship  \
0  243865.2644                   0.0           

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2087 entries, 0 to 2086
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2087 non-null   int64  
 1   Age                             2087 non-null   float64
 2   Height                          2087 non-null   float64
 3   Weight                          2087 non-null   float64
 4   family_history_with_overweight  2087 non-null   int64  
 5   FAVC                            2087 non-null   int64  
 6   FCVC                            2087 non-null   float64
 7   NCP                             2087 non-null   float64
 8   CAEC                            2087 non-null   int64  
 9   SMOKE                           2087 non-null   int64  
 10  CH2O                            2087 non-null   float64
 11  SCC                             2087 non-null   int64  
 12  FAF                             20

In [25]:

cols = df.columns


prediction_col = 'NObeyesdad'
feature_cols = [c for c in cols if c != prediction_col]
X = df[feature_cols]
y = df[prediction_col]

In [33]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [27]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train[['Weight', 'BMI', 'Water_Calories_Ratio', 'family_history_with_overweight']], y_train)

RandomForestClassifier(random_state=42)

In [47]:
accuracy = model.score(X_val[['Weight', 'BMI', 'Water_Calories_Ratio', 'family_history_with_overweight']], y_val)
print("RandomFrorestAccuracy", accuracy)

RandomFrorestAccuracy 0.9664268585131894


 indicates that your model is performing very well on the validation set. This suggests that the features you've chosen (such as Weight, BMI, Water_Calories_Ratio, and family history with overweight) are strong predictors for the obesity classification

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

selected_features = ['Weight', 'BMI', 'Water_Calories_Ratio', 'family_history_with_overweight']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

logreg_model = LogisticRegression(random_state=42, max_iter=500)
logreg_model.fit(X_train_scaled, y_train)
logreg_accuracy = logreg_model.score(X_val_scaled, y_val)
print("Logistic Regression Accuracy:", logreg_accuracy)


Logistic Regression Accuracy: 0.9088729016786571


In [74]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [75]:
features = ['Weight', 'BMI', 'Water_Calories_Ratio', 'family_history_with_overweight']
X_train_selected = X_train[features]
X_val_selected = X_val[features]

In [76]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train_selected, y_train)

DecisionTreeClassifier(random_state=42)

In [78]:
val_predictions = tree_model.predict(X_val_selected)
val_accuracy = accuracy_score(y_val, val_predictions)
print("Decision Tree Accuracy:", val_accuracy)

Decision Tree Accuracy: 0.9568345323741008


I analyzed the factors influencing obesity classification by training three models: Random Forest, Logistic Regression, and Decision Tree. Using features like 'Weight', 'BMI', 'Water_Calories_Ratio', and 'family_history_with_overweight', I found that the Random Forest model achieved the highest accuracy of 96.64%, while Logistic Regression reached 90.89% and Decision Tree scored 95.68%. This shows that the selected features are crucial for predicting obesity levels, with Random Forest being the most effective model.

I decided to try other features with the Decision Tree model, as it also achieved a high accuracy. I experimented with different combinations of features to see if they would improve performance. 

In [64]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [65]:
features = ['Age', 'SMOKE', 'CALC', 'Water_Calories_Ratio', 'CAEC_IMC_Relationship']
X_train_selected = X_train[features]
X_val_selected = X_val[features]
X_test_selected = X_test[features]

In [67]:
tree_model_new = DecisionTreeClassifier(random_state=42)
tree_model_new.fit(X_train_new, y_train_new)

DecisionTreeClassifier(random_state=42)

In [68]:
val_predictions_new = tree_model_new.predict(X_val_new)
val_accuracy_new = accuracy_score(y_val_new, val_predictions_new)
print("Decision Tree Accuracy with new features:", val_accuracy_new)

Decision Tree Accuracy with new features: 0.8968824940047961


In [59]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [60]:
features = ['FCVC', 'NCP', 'CH2O', 'FAF', 'BMI', 'Alcohol_Smoke_Ratio']
X_train_selected = X_train[features]
X_val_selected = X_val[features]

In [61]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train_selected, y_train)

DecisionTreeClassifier(random_state=42)

In [63]:
val_predictions = tree_model.predict(X_val_selected)
val_accuracy = accuracy_score(y_val, val_predictions)
print("Decision Tree Accuracy with new features2", val_accuracy)

Decision Tree Accuracy with new features2 0.9640287769784173


First, I selected features like 'Age', 'SMOKE', 'CALC', 'Water_Calories_Ratio', and 'CAEC_IMC_Relationship', but this combination resulted in a lower accuracy of 89.69%. Then, I tried another set of features, including 'FCVC', 'NCP', 'CH2O', 'FAF', 'BMI', and 'Alcohol_Smoke_Ratio', which improved the accuracy to 96.40%. Despite this, the Random Forest model still achieved the highest accuracy overall. To further explore, I decided to experiment with other features in the Random Forest model as well.

In [90]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(X_train[['Age', 'SMOKE', 'CALC', 'Water_Calories_Ratio', 'CAEC_IMC_Relationship']], y_train)


RandomForestClassifier(random_state=42)

In [84]:
accuracy = model.score(X_val[['Age', 'SMOKE', 'CALC', 'Water_Calories_Ratio', 'CAEC_IMC_Relationship']], y_val)
print("RandomFrorestAccuracy2", accuracy)

RandomFrorestAccuracy2 0.9136690647482014


In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_selected, y_train)


RandomForestClassifier(random_state=42)

In [85]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [86]:
features_chosen_set = ['FCVC', 'NCP', 'CH2O', 'FAF', 'BMI', 'Alcohol_Smoke_Ratio']
X_train_selected = X_train[features_chosen_set]
X_val_selected = X_val[features_chosen_set]

In [87]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_selected, y_train)


RandomForestClassifier(random_state=42)

In [88]:
val_accuracy = model.score(X_val_selected, y_val)
print(f"Random Forest Accuracy with chosen features: {val_accuracy}")

Random Forest Accuracy with chosen features: 0.9784172661870504


In [82]:
accuracy = model.score(X_val[['FCVC', 'NCP', 'CH2O', 'FAF', 'BMI', 'Alcohol_Smoke_Ratio']], y_val)
print("RandomFrorestAccuracy2", accuracy)

RandomFrorestAccuracy2 0.9784172661870504


In [89]:
import joblib
joblib.dump(model, 'random_forest_model_with_selected_features.pkl')

['random_forest_model_with_selected_features.pkl']

I experimented with different feature sets using the Random Forest model. First, I used features like 'Age', 'SMOKE', 'CALC', 'Water_Calories_Ratio', and 'CAEC_IMC_Relationship', which resulted in an accuracy of 91.37%. Then, I tried another set of features, including 'FCVC', 'NCP', 'CH2O', 'FAF', 'BMI', and 'Alcohol_Smoke_Ratio', which significantly improved the accuracy to 97.84%. This turned out to be the highest accuracy, so I chose this feature set and saved the model with these features for future use.